In [ ]:
import sys
sys.path.append('/home/mathieu/software/pysal')
import pysalvador as sal
import pandas as pd
import numpy as np
import itertools
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.patches import FancyArrow
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.ticker import LogLocator
from PIL import Image
import seaborn as sns
sns.set(style='ticks', font='DejaVu Sans')

In [ ]:
#path for figures
fig_path = '/home/mathieu/mhenault_landrylab/Publications/lrma/resubmission1_elife/fig/'

## Import haploids metadata

In [ ]:
haploids = pd.read_excel('/home/mathieu/mhenault_landrylab/Experiments/transposition_assay_2022/strains_crosses_3.ods',
                         sheet_name='haploids', header=None, index_col=0)
haploids.columns = ['strain','genotype','description','','mat','media','box','pos','date','person','ref']
haploids['background'] = haploids['strain'].apply(lambda x: x.split(' ')[0])
cross_plate = pd.read_csv('/home/mathieu/mhenault_landrylab/Experiments/transposition_assay_2022/crosses_plate.csv', index_col=0)
cn_ty1 = pd.read_csv('/home/mathieu/mhenault_landrylab/Experiments/transposition_assay_2021/cn_ty1.csv')
cn_ty1.loc[1, 'strain'] = 'UWOPS-79-140'
cn_ty1.index = cn_ty1['strain'].values

backgrounds = ['YPS744','UWOPS-79-140','UWOPS-91-202','GB23','LL17-69','PPC6','VC2','D11','GB20','LL17-02','PPC3','SQ107','IO33','PPC20']
background_alias = dict(zip(backgrounds, ['YPS744','UWOPS-79-140','UWOPS-91-202','MSH-604','LL2012_014','LL2012_021','LL2012_028','LL2011_012','MSH-587-1',\
                             'LL2012_011','LL2012_019','LL2011_009','LL2011_005','LL2012_016']))
bgd_color = dict(zip(backgrounds, ["#00CD00","#EE0000","#EE0000","#EE0000","#EE0000","#EE0000","#EE0000","#00008B","#00008B","#00008B","#00008B","#00008B","#912CEE","#912CEE"]))
bgd_group = dict(zip(backgrounds, ["SpA","SpB","SpB","SpB","SpB","SpB","SpB","SpC","SpC","SpC","SpC","SpC","SpC*","SpC*"]))
group_alias = {'SpA':r'$SpA$','SpB':r'$SpB$','SpC':r'$SpC$','SpC*':r'$SpC^*$'}


hybrids = [frozenset(['YPS744']),
           frozenset(['UWOPS-79-140']),
           frozenset(['D11']),
           frozenset(['YPS744', 'UWOPS-79-140']),
           frozenset(['YPS744', 'D11']),
           frozenset(['D11', 'UWOPS-79-140'])]
           
hybrid_colors = dict(zip(hybrids, ["#00CD00",
                                   "#EE0000",
                                   "#00008B", 
                                   np.array([168, 145, 0])/255, 
                                   np.array([0, 145, 98])/255,
                                   np.array([168, 0, 98])/255]
                                   ))

for bgd, df in haploids.groupby('background'):
    if bgd in cn_ty1.index:
        haploids.loc[df.index, 'cn'] = cn_ty1.loc[bgd,'median_corr']
haploids['group'] = haploids['background'].apply(lambda x: bgd_group[x])

## Import hybrids metadata

In [ ]:
cross_rho0 = pd.read_excel('/home/mathieu/mhenault_landrylab/Experiments/transposition_assay_2022/strains_crosses_3.ods',
                         sheet_name='cross_rho0', header=0, index_col=0).set_index('hybrid_name')

In [ ]:
# export plate ploidy 1

ploidy1 = cross_rho0.loc[cross_rho0['plate_ploidy']==1].copy().fillna('')
ploidy1['value'] = ploidy1.apply(lambda x: f'{x["src_well"]}.{x["colony"]}', axis=1)
ploidy1.pivot_table(index='row', columns='col', values='value', aggfunc=lambda x: x)\
.to_csv('/home/mathieu/mhenault_landrylab/Experiments/transposition_assay_2022/ploidy1.csv')

In [ ]:
def rates_table(results, meta):
    
    rates = []

    wells = results.index[5:]

    for i in results.columns:
        s, plasmid, date, count, N = results.loc[['strain_id', 'plasmid', 'date', 'count', 'N'], i]
        counts = results.loc[wells, i]
        counts = counts[~np.isnan(counts.astype(np.float32))].astype(np.int32)

        if counts.shape[0] > 3:

            if ~np.all(counts==0):
                upper_bound = False
            else:
                counts[0] = 1
                upper_bound = True

            n = sal.newtonLD(counts)
            ci = sal.confintLD(counts, show_iter=False)

            rates.append([s, plasmid, date, count, N, n, ci[0], ci[1], upper_bound])

    rates = pd.DataFrame(rates, columns=['strain_id', 'plasmid', 'date', 'count', 'N', 'n', 'n_0.025', 'n_0.975', 'upper_bound'])
    rates['rate'] = rates['n']/rates['N']
    rates['rate_0.025'] = rates['n_0.025']/rates['N']
    rates['rate_0.975'] = rates['n_0.975']/rates['N']
    rates.set_index('strain_id', inplace=True)

    # append metadata
    for c in meta.columns:
        rates[c] = meta.loc[rates.index, c].values
        
    return rates.reset_index()

# Haploids

In [ ]:
results_haploids = pd.read_excel('/home/mathieu/mhenault_landrylab/Experiments/transposition_assay_2022/data_2023/results.xlsx',
                                 sheet_name='haploids', header=None, index_col=0)
rates_haploids = rates_table(results_haploids, haploids)

## Fig 5

In [ ]:
fig, ax = plt.subplots(figsize=[6,5], gridspec_kw=dict(right=0.95, top=0.75, bottom=0.12, left=0.2))

ylim = [1e-10, 3e-6]

for i in rates_haploids.index:
    bgd, plasmid, group, rate, rate_0025, rate_0975, upper, cn = \
    rates_haploids.loc[i, ['background', 'plasmid', 'group', 'rate', 'rate_0.025', 'rate_0.975', 'upper_bound', 'cn']]
    
    mec = bgd_color[bgd]
    
    fc = 'magenta'
    if upper:
        fc = 'white'
        
    else:
        fc = mec
        
    m = 'x'
    if plasmid == 'SpB':
        m = 'o'
        x = cn*0.98
    elif plasmid == 'SpC':
        m = 's'
        x = cn*1.02
        
    if not upper:
        ax.plot([x, x], [rate_0025, rate_0975], color='k', zorder=1)
    ax.scatter(x, rate, marker=m, s=36, color=fc, edgecolors=mec, zorder=2)
    
bgd_rate_cn = rates_haploids.groupby(['background', 'cn'])['rate'].min().reset_index().sort_values(by='cn').set_index('background')
bgd_rate_cn['cn_rank'] = np.logspace(-5, -7, 8)

#add background labels
"""for bgd in bgd_rate_cn.index:
    cn, rate, rank = bgd_rate_cn.loc[bgd]
    
    fa = FancyArrow(cn, ylim[0], 0, ylim[1]-ylim[0], width=0.15*cn, head_length=0, head_width=0,
                    fc='0.5', lw=0, alpha=0.3, zorder=0)
    ax.add_patch(fa)
    if rate > 7e-8:
        y = 3e-9
    else:
        y = 3e-7
    ax.text(cn, y, background_alias[bgd], rotation=90, va='center', ha='center', size=8)"""
    
ax.set_ylim(ylim)
ax.set_yscale('log')
ax.set_ylabel('Transposition rate\n(cell$^{-1}$ gen$^{-1}$)')
ax.set_xlabel('Chromosomal Ty1 CN')
ax.set_xscale('log')
ax.set_xticks([1,2,5,10,20, 30])
ax.set_xticklabels([1,2,5,10,20, 30])

legend_upper = 1.37

legend_elms0 = [Line2D([0], [0], color='w', marker='o', mfc=fc, ms=8, label=group_alias[l]) for (l, fc) in zip(['SpA','SpB','SpC'], ['#00CD00', '#EE0000', '#00008B'])]
legend0 = plt.legend(handles=legend_elms0, title='Haploid background', bbox_to_anchor=[-0.25, legend_upper], loc='upper left')

legend_elms1 = [Line2D([0], [0], color='w', marker=m, mfc='k', mec='k', ms=8, label=group_alias[l]) for (l, m) in zip(['SpB','SpC'], ['o', 's'])]
legend1 = plt.legend(handles=legend_elms1, title='Ty1 plasmid', bbox_to_anchor=[0.3, legend_upper], loc='upper left')

legend_elms2 = [Line2D([0], [0], color='w', marker='o', mfc=fc, mec='k', ms=8, label=l) for (l, fc) in zip(['No','Yes'], ['k', 'w'])]
legend2 = plt.legend(handles=legend_elms2, title='Upper bound', bbox_to_anchor=[0.7, legend_upper], loc='upper left')

for legend in [legend0, legend1, legend2]:
    ax.add_artist(legend)


sns.despine()

#for ext in ['svg', 'png']:
#    plt.savefig(f'{fig_path}Fig5.{ext}', dpi=300)
    
#plt.show()
plt.close()

In [ ]:
rates_old = pd.read_csv('/home/mathieu/mhenault_landrylab/Experiments/transposition_assay_2021/res/rates_061221.csv', index_col=0)
rates_old.set_index(['cross','plasmid'], inplace=True)

In [ ]:
for (p1, p2, plasmid), df in rates.groupby(['parent1', 'parent2', 'plasmid']):
    key1 = (f'{p1}.{p2}', plasmid[-1])
    key2 = (f'{p2}.{p1}', plasmid[-1])
    
    if key1 in rates_old.index:
        ro = rates_old.loc[key1, 'rate']
        rates.loc[df.index, 'rate_old'] = ro
        
    elif key2 in rates_old.index:
        ro = rates_old.loc[key2, 'rate']
        rates.loc[df.index, 'rate_old'] = ro

In [ ]:
sns.scatterplot(x='rate', y='rate_old', data=rates)
plt.show()
plt.close()

# Hybrids

In [ ]:
results_hybrids = pd.read_excel('/home/mathieu/mhenault_landrylab/Experiments/transposition_assay_2022/data_2023/results.xlsx',
                                 sheet_name='hybrids', header=None, index_col=0)
rates_hybrids = rates_table(results_hybrids, cross_rho0)

In [ ]:
# create dictionary of crosses labels for plotting
mat_alias = {'a': r'$a$', 'alpha':r'$\alpha$'}
rho_alias = {'rho+': r'$\rho^+$', 'rho0':r'$\rho^0$'}

hybrid_labels_rho = {}

for idx, df in rates_hybrids.groupby(['parent1_background', 'parent1_mat', 'parent1_rho', 'parent2_background', 'parent2_mat', 'parent2_rho']):
    (bgd1, mat1, rho1, bgd2, mat2, rho2) = idx

    times = r'$\times$'
    label = f"{background_alias[bgd2]} $MAT${mat_alias[mat2]} {rho_alias[rho2]}{times}\n{background_alias[bgd1]} $MAT${mat_alias[mat1]} {rho_alias[rho1]}"
    
    hybrid_labels_rho[idx] = label
    
hybrid_labels_mat ={}
for idx, df in rates_hybrids.groupby(['parent1_background', 'parent1_mat', 'parent2_background', 'parent2_mat']):
    (bgd1, mat1, bgd2, mat2) = idx

    times = r'$\times$'
    label = f"{background_alias[bgd2]} $MAT${mat_alias[mat2]} ({group_alias[bgd_group[bgd2]]}){times}\n{background_alias[bgd1]} $MAT${mat_alias[mat1]} ({group_alias[bgd_group[bgd1]]})"
    
    hybrid_labels_mat[idx] = label
    
hybrid_labels ={}
for idx, df in rates_hybrids.groupby(['parent1_background', 'parent1_mat', 'parent2_background', 'parent2_mat']):
    (bgd1, mat1, bgd2, mat2) = idx

    times = r'$\times$'
    label = f"{background_alias[bgd2]} ({group_alias[bgd_group[bgd2]]}){times}\n{background_alias[bgd1]} ({group_alias[bgd_group[bgd1]]})"
    
    hybrid_labels[idx] = label

## Fig S13

In [ ]:
def plot_cross_parents(df, s, X, ax, dx=0.2, color='k'):
    
    global hybrid_labels_rho
    
    for plasmid in ['SpB', 'SpC']:
    
        rate, rate_0025, rate_0975, upper = df.loc[plasmid, ['rate', 'rate_0.025', 'rate_0.975', 'upper_bound']]
        
        fc = 'magenta'
        if upper:
            fc = 'white'

        else:
            fc = color
        
        m = 'x'
        x = np.nan
        if plasmid == 'SpB':
            m = 'o'
            x =  X-dx
            
        elif plasmid == 'SpC':
            m = 's'
            x = X+dx

        if not upper:
            ax.plot([x, x], [rate_0025, rate_0975], color='k', zorder=1)
        ax.scatter(x, rate, marker=m, s=48, color=fc, edgecolors=color, zorder=2)
        
    idx = df.value_counts(['parent1_background', 'parent1_mat', 'parent1_rho', 'parent2_background', 'parent2_mat', 'parent2_rho']).index[0]
    
    return hybrid_labels_rho[idx]

In [ ]:
fig, axes = plt.subplots(figsize=[10,7], ncols=3, 
                        gridspec_kw=dict(left=0.1, right=0.97, top=0.7, bottom=0.3, wspace=0.35))
    
hyb_included = {hyb:np.nan for hyb in hybrids}

for (bgd1, bgd2), ax_idx in zip(itertools.combinations(['YPS744', 'UWOPS-79-140', 'D11'], 2), range(3)):

    #bgd1, bgd2 = sorted([bgd1, bgd2])
    
    ax = axes[ax_idx]
    x_labels = {}
    
    for i, (p1, p2) in zip(range(3),
                              [(bgd1, bgd1), (bgd1, bgd2), (bgd2, bgd2)]):
        
        df = rates_hybrids.loc[(rates_hybrids['parent1_background']==p2) & (rates_hybrids['parent2_background']==p1)]
        hyb = frozenset([p1, p2])
        
        idx = tuple(df.iloc[0][['parent1_background', 'parent1_mat', 'parent2_background', 'parent2_mat']])    
        hyb_included[hyb] = idx
        
        # split in hybrid genotypes
        
        #extract how many genotypes exist for that hybrid
        uhyb = df.sort_values(by='parent1_rho')['strain_id'].unique()
        nuhyb = uhyb.shape[0]
        #determine plotting order of the different crosses
        if i == 1:
            s_order = dict(zip(uhyb, np.arange(nuhyb)))
        elif i == 0:
            s_order = dict(zip(uhyb, (np.arange(nuhyb)+1)*(-1)))
        elif i == 2:
            s_order = dict(zip(uhyb, np.arange(nuhyb)+2))       
        
        # get the color of the cross
        color = hybrid_colors[frozenset([p1, p2])]
        
        for s, df1 in df.groupby('strain_id'):
            df1 = df1.set_index('plasmid')
            
            x = s_order[s]
            l = plot_cross_parents(df1, s, x, ax, dx=0.1, color=color)
            x_labels[x] = l
            
    x_ticks = sorted(x_labels.keys())
    ax.set_xticks(x_ticks)
    ax.set_xticklabels([x_labels[x] for x in x_ticks], rotation=90, ha='center', va='top', size=10)
    
    ax.set_yscale('log')
    ax.set_ylim(5e-11, 2e-6)
    if ax_idx == 0:
        ax.set_ylabel('Transposition rate (cell$^{-1}$ gen$^{-1}$)')
    
    sns.despine(ax=ax)
    
# draw legends
ax = fig.add_axes([0,1,1,0])
ax.axis('off')

legend_elms0 = [Line2D([0], [0], color='w', marker='o', mfc=hybrid_colors[k], ms=10, 
                        label=hybrid_labels[idx]) for k,idx in hyb_included.items()]
legend0 = plt.legend(handles=legend_elms0, title='Diploid background', bbox_to_anchor=[0, 1], loc='upper left', ncol=2)

legend_elms1 = [Line2D([0], [0], color='w', marker=m, mfc='k', mec='k', ms=10, label=group_alias[l]) for (l, m) in zip(['SpB','SpC'], ['o', 's'])]
legend1 = plt.legend(handles=legend_elms1, title='Ty1 plasmid', bbox_to_anchor=[0.6, 1], loc='upper left')

legend_elms2 = [Line2D([0], [0], color='w', marker='o', mfc=fc, mec='k', ms=10, label=l) for (l, fc) in zip(['No','Yes'], ['k', 'w'])]
legend2 = plt.legend(handles=legend_elms2, title='Upper bound', bbox_to_anchor=[0.85, 1], loc='upper left')

for legend in [legend0, legend1, legend2]:
    ax.add_artist(legend)
    
#for ext in ['svg','png']:
#    plt.savefig(f'{fig_path}Supp_FigS12.{ext}', dpi=300)
#plt.show()
plt.close()

# Correlations between rates for $\rho^0$ reciprocal crosses
## Fig 7

In [ ]:
#fig, ax = plt.subplots(figsize=[8,4.5], gridspec_kw=dict(left=0.15, bottom=0.2, right=0.55, top=0.93))
fig = plt.figure(figsize=[8, 4.5])
gs = plt.GridSpec(ncols=1, nrows=1, left=0.15, bottom=0.2, right=0.55, top=0.93)
ax = fig.add_subplot(gs[0])

#hyb_included = {}
hyb_included = {hyb:np.nan for hyb in hybrids}
xlim = np.array([7e-11, 2e-7])

for hyb in hybrids:
    Hyb = sorted(list(hyb))
    if len(Hyb) == 1:
        Hyb = [Hyb[0], Hyb[0]]
    
    df = rates_hybrids.loc[(rates_hybrids['parent1_background']==Hyb[0]) & (rates_hybrids['parent2_background']==Hyb[1])]
    
    if df['parent1_rho'].unique().shape[0] > 1:
        
        
        for plasmid, df1 in df.sort_values(by='parent1_rho').groupby('plasmid'):
            
            rate1, rate1_0025, rate1_0975, rate1_upper = df1.iloc[0][['rate', 'rate_0.025', 'rate_0.975', 'upper_bound']]
            rate2, rate2_0025, rate2_0975, rate2_upper = df1.iloc[1][['rate', 'rate_0.025', 'rate_0.975', 'upper_bound']]
            
            idx = tuple(df1.iloc[0][['parent1_background', 'parent1_mat', 'parent2_background', 'parent2_mat']])
            
            hyb_included[hyb] = idx
            
            if plasmid == 'SpB':
                m = 'o'

            elif plasmid == 'SpC':
                m = 's'
                
            
            if any([rate1_upper, rate2_upper]):
                mfc = 'w'
            else:
                mfc = hybrid_colors[hyb]
            
            ax.scatter(rate2, rate1, marker=m, zorder=2, color=mfc, edgecolors=hybrid_colors[hyb])
            
            if not rate2_upper:
                ax.plot([rate2_0025, rate2_0975], [rate1, rate1], color='k', zorder=1)
            if not rate1_upper:
                ax.plot([rate2, rate2], [rate1_0025, rate1_0975], color='k', zorder=1)

ax.set_xscale('log')
ax.set_xlim(xlim)
ax.set_xlabel('Transposition rate (cell$^{-1}$ gen$^{-1}$)\n$MAT$'+mat_alias["a"]+' parent mtDNA')

ax.set_yscale('log')
ax.set_ylim(xlim)
ax.set_ylabel('Transposition rate (cell$^{-1}$ gen$^{-1}$)\n$MAT$'+mat_alias["alpha"]+' parent mtDNA')

ax.plot(xlim, xlim, zorder=0, color='k', lw=0.5)

# Legends
legend_left = 1.1

legend_elms0 = [Line2D([0], [0], color='w', marker='o', mfc=hybrid_colors[k], ms=10, 
                       label=hybrid_labels_mat[idx]) for k,idx in hyb_included.items() if type(idx)==tuple]
legend0 = plt.legend(handles=legend_elms0, title='Diploid background', bbox_to_anchor=[legend_left, 1.1], loc='upper left', ncol=1)

legend_elms1 = [Line2D([0], [0], color='w', marker=m, mfc='k', mec='k', ms=8, label=l) for (l, m) in zip(['$SpB$','$SpC$'], ['o', 's'])]
legend1 = plt.legend(handles=legend_elms1, title='Ty1 plasmid', bbox_to_anchor=[legend_left, 0.43], loc='upper left')

legend_elms2 = [Line2D([0], [0], color='w', marker='o', mfc=fc, mec='k', ms=8, label=l) for (l, fc) in zip(['No','Yes'], ['k', 'w'])]
legend2 = plt.legend(handles=legend_elms2, title='Upper bound', bbox_to_anchor=[legend_left, 0.16], loc='upper left')

for legend in [legend0, legend1, legend2]:
    ax.add_artist(legend)

sns.despine()

# add new axis for labels
ax1 = fig.add_subplot(gs[0])
ax1.axis('off')

dxy = 0.15
for hyb in hybrids:
    if len(hyb) == 2:
        Hyb = sorted(list(hyb))
        fa = FancyArrow(dxy, dxy, 0.1, -0.1, width=0.08, lw=0, length_includes_head=True, zorder=-1,
                       head_length=0.02, head_width=0.08, fc=hybrid_colors[hyb], alpha=0.7)
        ax1.add_patch(fa)
        ax1.text(dxy+0.05, dxy-0.05, group_alias[bgd_group[Hyb[1]]], rotation=315, ha='center', va='center', size=10, c='w')
        
        fa = FancyArrow(dxy, dxy, -0.1, +0.1, width=0.08, lw=0, length_includes_head=True, zorder=-1,
                       head_length=0.02, head_width=0.08, fc=hybrid_colors[hyb], alpha=0.7)
        ax1.add_patch(fa)
        ax1.text(dxy-0.05, dxy+0.05, group_alias[bgd_group[Hyb[0]]], rotation=315, ha='center', va='center', size=10, c='w')
        
        dxy += 0.065

#for ext in ['svg', 'png']:
#    plt.savefig(f'{fig_path}Fig7.{ext}', dpi=300)

#plt.show()
plt.close()

# Correlations between rates for Ty1 plasmids
## Fig 6

In [ ]:
fig, axes = plt.subplots(nrows=2, figsize=[7.5,8], 
                         gridspec_kw=dict(left=0.16, bottom=0.1, right=0.6, top=0.94, hspace=0.4))

## Haploids

ax = axes[0]
xlim = np.array([5e-11, 3e-6])


for bgd, df in rates_haploids.groupby('background'):

    if df['plasmid'].unique().shape[0] > 1:
        
        df = df.set_index('plasmid')
        
        b_rate, b_0025, b_0975, b_upper = df.loc['SpB', ['rate', 'rate_0.025', 'rate_0.975', 'upper_bound']]
        c_rate, c_0025, c_0975, c_upper = df.loc['SpC', ['rate', 'rate_0.025', 'rate_0.975', 'upper_bound']]
        
        if any([b_upper, c_upper]):
            mfc = 'w'
        else:
            mfc = bgd_color[bgd]
        
        ax.scatter(b_rate, c_rate, marker='o', color=mfc, edgecolors=bgd_color[bgd], zorder=2)
        
        if not b_upper:
            ax.plot([b_0025, b_0975], [c_rate, c_rate], color='k', zorder=1)
        if not c_upper:
            ax.plot([b_rate, b_rate], [c_0025, c_0975], color='k', zorder=1)
    
ax.set_xscale('log')
ax.set_xlim(xlim)
ax.set_xlabel('Transposition rate (cell$^{-1}$ gen$^{-1}$)\n'+group_alias['SpB']+' Ty1 variant')

ax.set_yscale('log')
ax.set_ylim(xlim)
ax.set_ylabel('Transposition rate (cell$^{-1}$ gen$^{-1}$)\n'+group_alias['SpC']+' Ty1 variant')

ax.plot(xlim, xlim, zorder=0, color='k', lw=0.5)
    
## Hybrids

hyb_included = {hyb:np.nan for hyb in hybrids}

ax = axes[1]
for (bgd1, mat1, rho1, bgd2, mat2, rho2), df in rates_hybrids.groupby(['parent1_background', 'parent1_mat', 'parent1_rho',
                                                                      'parent2_background', 'parent2_mat', 'parent2_rho']):

    if df['plasmid'].unique().shape[0] > 1:
        
        df = df.set_index('plasmid')
        hyb = frozenset([bgd1, bgd2])
        
        b_rate, b_0025, b_0975, b_upper = df.loc['SpB', ['rate', 'rate_0.025', 'rate_0.975', 'upper_bound']]
        c_rate, c_0025, c_0975, c_upper = df.loc['SpC', ['rate', 'rate_0.025', 'rate_0.975', 'upper_bound']]
        
        idx = tuple(df.iloc[0][['parent1_background', 'parent1_mat', 'parent2_background', 'parent2_mat']])
            
        hyb_included[hyb] = idx
        
        if any([b_upper, c_upper]):
            mfc = 'w'
        else:
            mfc = hybrid_colors[hyb]
        
        ax.scatter(b_rate, c_rate, marker='o', color=mfc, edgecolors=hybrid_colors[hyb], zorder=2)
        
        if not b_upper:
            ax.plot([b_0025, b_0975], [c_rate, c_rate], color='k', zorder=1)
        if not c_upper:
            ax.plot([b_rate, b_rate], [c_0025, c_0975], color='k', zorder=1)
    

ax.set_xscale('log')
ax.set_xlim(xlim)
ax.set_xlabel('Transposition rate (cell$^{-1}$ gen$^{-1}$)\n'+group_alias['SpB']+' Ty1 variant')

ax.set_yscale('log')
ax.set_ylim(xlim)
ax.set_ylabel('Transposition rate (cell$^{-1}$ gen$^{-1}$)\n'+group_alias['SpC']+' Ty1 variant')

ax.plot(xlim, xlim, zorder=0, color='k', lw=0.5)

# Add legends

ax = fig.add_axes([0.6, 0, 0.4, 1])
ax.axis('off')

legend_elms0 = [Line2D([0], [0], color='w', marker='o', mfc=fc, ms=10, label=group_alias[l]) for (l, fc) in zip(['SpA','SpB','SpC'], ['#00CD00','#EE0000', '#00008B'])]
legend0 = plt.legend(handles=legend_elms0, title='Haploid background', bbox_to_anchor=[0.1, 0.85], loc='upper left')

legend_elms1 = [Line2D([0], [0], color='w', marker='o', mfc=fc, mec='k', ms=8, label=l) for (l, fc) in zip(['No','Yes'], ['k', 'w'])]
legend1 = plt.legend(handles=legend_elms1, title='Upper bound', bbox_to_anchor=[0.1, 0.7], loc='upper left')

legend_elms2 = [Line2D([0], [0], color='w', marker='o', mfc=hybrid_colors[k], ms=10, 
                       label=hybrid_labels[idx]) for k,idx in hyb_included.items()]
legend2 = plt.legend(handles=legend_elms2, title='Diploid background', bbox_to_anchor=[0.1, 0.5], loc='upper left', ncol=1)

for legend in [legend0, legend1, legend2]:
    ax.add_artist(legend)

fig.text(0.02, 0.96, 'A', size=24, fontweight='bold', font='Arial')
fig.text(0.02, 0.46, 'B', size=24, fontweight='bold', font='Arial')
    
    
sns.despine()

#for ext in ['svg', 'png']:
#    plt.savefig(f'{fig_path}Fig6.{ext}')
#plt.show()
plt.close()